* Задание 1
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('keywords.csv')

In [4]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [5]:
def region(row):
    b = 'Центр'
    a = 'Северо-Запад'
    c = 'Дальний Восток'
    f = 'undefined'
    if geo_data['Центр'][0] in row['keyword']:
        return b
    if geo_data['Центр'][1] in row['keyword']:
        return b
    if geo_data['Центр'][2] in row['keyword']:
        return b
    if geo_data['Северо-Запад'][0] in row['keyword']:
        return a
    if geo_data['Северо-Запад'][1] in row['keyword']:
        return a
    if geo_data['Северо-Запад'][2] in row['keyword']:
        return a
    if geo_data['Дальний Восток'][0] in row['keyword']:
        return c
    if geo_data['Дальний Восток'][1] in row['keyword']:
        return c
    if geo_data['Дальний Восток'][2] in row['keyword']:
        return c
    else:
        return f

In [6]:
data['region'] = data.apply(region, axis=1)
filtered_region = data[ (data['region']=='Центр') | (data['region']=='Северо-Запад') | (data['region']=='Дальний Восток')]

 * Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [7]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
joined = ratings.merge(movies, on='movieId', how='left')
filtered = joined.filter(items = ['movieId','rating', 'title'])

In [10]:
def movie_class(row):
    a = 'низкий рейтинг'
    b = 'средний рейтинг'
    c = 'высокий рейтинг'
    if row['rating']<=2:
        return a
    if 2<row['rating']<=4:
        return b
    if 4.5<=row['rating']<=5:
        return c

In [11]:
filtered['class'] = filtered.apply(movie_class, axis = 1)

 * Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [12]:
from datetime import datetime

In [13]:
ratings_dz_3 = ratings = pd.read_csv('ratings.csv')

In [14]:
group_user = ratings_dz_3.groupby('userId').count().reset_index()

In [15]:
over_100_rating = group_user[group_user['rating']>100]
to_list = over_100_rating['userId'].tolist()
filtered_ratings_dz_3 = ratings_dz_3[ratings_dz_3['userId'].isin(to_list)]

In [16]:
ratings_timestamp = filtered_ratings_dz_3.groupby('userId').agg([min, max])

In [17]:
ratings_timestamp['difference'] = ratings_timestamp['timestamp']['max'] - ratings_timestamp['timestamp']['min']

In [18]:
unix_time = ratings_timestamp['difference'].mean()

In [19]:
value = datetime.fromtimestamp(unix_time)
print(value.strftime('%d'))

10


 * Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [20]:
movies_dz_4 = pd.read_csv('movies.csv')
ratings_dz_4 = pd.read_csv('ratings.csv')

In [21]:
# В переменную years запишите список из всех годов с 1950 по 2010.
years = [x for x in range(1950,2010+1)]

In [22]:
def production_year(row):
    for i in range(len(years)):
        m = '1900'
        # достаем из строки год
        u = row['title'].split(' ')[-1].strip('()')
        if u in str(years[i]):
            return u
    return m

In [23]:
movies_dz_4['year'] = movies_dz_4.apply(production_year, axis = 1)

In [24]:
joined_dz_4 = movies_dz_4.merge(ratings_dz_4, how = 'left', on = 'movieId')

In [25]:
filtered_join = joined_dz_4.filter(items = ['movieId','title', 'rating','year'])
filtered_join.head()

,movieId,title,rating,year
0,1,Toy Story (1995),3.0,1995
1,1,Toy Story (1995),4.0,1995
2,1,Toy Story (1995),5.0,1995
3,1,Toy Story (1995),2.0,1995
4,1,Toy Story (1995),3.0,1995


In [ ]:
final_df = filtered_join.groupby('year').mean().sort_values(by=['rating', 'year'],ascending = [False,False])
final_df.filter(items = ['year','rating']).head()

In [ ]:
# тоже интересная статистика по годам, изменение рейтинга
final_df_year = filtered_join.groupby('year').mean().sort_values(by=['year', 'rating'],ascending = [False,False])
final_df_year.head()